In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
from datetime import datetime
import numpy as np

class MySQLDataLoader:
    def __init__(self, user, password, database, host="localhost"):
        self.connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
    
    def get_max_fecha_extraccion(self):
        cursor = self.connection.cursor()
        cursor.execute("SELECT MAX(fecha_extraccion) FROM coches_en_venta;")
        result = cursor.fetchone()
        cursor.close()
        return result[0] if result[0] else None
    
    def get_concesionario_count(self):
        cursor = self.connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM concesionario;")
        result = cursor.fetchone()
        cursor.close()
        return result[0]
    
    def insert_ignore(self, table, column, value):
        cursor = self.connection.cursor()
        query = f"""
        INSERT INTO {table} ({column})
        SELECT %s
        WHERE NOT EXISTS (
            SELECT {column} FROM {table} WHERE {column} = %s
        )"""
        cursor.execute(query, (value, value))
        if table == "modelo_titulo":
            campo_id = "modelo"
        else:
            campo_id = table
        cursor.execute(f"SELECT {campo_id}_id FROM {table} WHERE {column} = %s;", (value,))
        result = cursor.fetchone()
        cursor.close()
        return result[0]
    
    def insert_modelo_titulo(self, marca_id, nombre_modelo):
        cursor = self.connection.cursor()
        query = """
        INSERT INTO modelo_titulo (marca_id, nombre_modelo)
        SELECT %s, %s
        WHERE NOT EXISTS (
            SELECT nombre_modelo FROM modelo_titulo WHERE nombre_modelo = %s AND marca_id = %s
        )"""
        cursor.execute(query, (marca_id, nombre_modelo, nombre_modelo, marca_id))
        cursor.execute("SELECT modelo_id FROM modelo_titulo WHERE nombre_modelo = %s AND marca_id = %s;", (nombre_modelo, marca_id))
        result = cursor.fetchone()
        cursor.close()
        return result[0]
    
    def insert_url(self, url):
        cursor = self.connection.cursor()
        query = """
        INSERT INTO urls (url)
        VALUES (%s)
        ON DUPLICATE KEY UPDATE url = VALUES(url);"""
        cursor.execute(query, (url,))
        cursor.execute("SELECT url_id FROM urls WHERE url = %s;", (url,))
        result = cursor.fetchone()
        cursor.close()
        return result[0]
    
    def insert_ruta_imagen(self, ruta_imagen):
        cursor = self.connection.cursor()
        query = """
        INSERT INTO ruta_imagen (ruta_imagen)
        VALUES (%s)
        ON DUPLICATE KEY UPDATE ruta_imagen = VALUES(ruta_imagen);"""
        cursor.execute(query, (ruta_imagen,))
        cursor.execute("SELECT ruta_imagen_id FROM ruta_imagen WHERE ruta_imagen = %s;", (ruta_imagen,))
        result = cursor.fetchone()
        cursor.close()
        return result[0]
    
    def get_concesionario_id(self, nombre_concesionario):
        cursor = self.connection.cursor()
        cursor.execute("SELECT concesionario_id FROM concesionario WHERE nombre_concesionario = %s;", (nombre_concesionario,))
        result = cursor.fetchone()
        cursor.close()
        return result[0] if result else None
        
    def process_csv_concesionarios(self, file_path):
        df = pd.read_csv(file_path)
        if len(df) > self.get_concesionario_count():
            for _, row in df.iterrows():
                query = """
                INSERT INTO concesionario (nombre_concesionario, calle, provincia_id, codigo_postal, municipio_id)
                VALUES (%s, %s, NULL, %s, NULL)
                ON DUPLICATE KEY UPDATE nombre_concesionario = VALUES(nombre_concesionario);"""
                cursor = self.connection.cursor()
                cursor.execute(query, (
                    row["nombre"],
                    row["calle"] if pd.notna(row["calle"]) else None,
                    int(row["codigo_postal"]) if pd.notna(row["codigo_postal"]) else None
                ))
                self.connection.commit()
                cursor.close()

    def process_csv_coches(self, file_path):
        df = pd.read_csv(file_path, parse_dates=["fecha_extraccion"], dtype={
            "puertas": "Int64", "mes_matricula": "Int64", "anio_matricula": "Int64"
        })
        max_fecha = self.get_max_fecha_extraccion()
        df = df[df["fecha_extraccion"] > max_fecha] if max_fecha else df

        anio_actual = datetime.now().year
        
        for _, row in df.iterrows():
            combustible_id = self.insert_ignore("combustible", "nombre_combustible", row["combustible"])
            marca_id = self.insert_ignore("marca", "nombre_marca", row["marca_sola"])
            modelo_id = self.insert_modelo_titulo(marca_id, row["modelo_titulo"])
            color_id = self.insert_ignore("color", "nombre_color", row["color"])
            carroceria_id = self.insert_ignore("carroceria", "nombre_carroceria", row["carroceria"])
            distintivo_ambiental_id = self.insert_ignore("distintivo_ambiental", "nombre_distintivo_ambiental", row["distintivo_ambiental"])
            url_id = self.insert_url(row["url"])
            ruta_imagen_id = self.insert_ruta_imagen(row["ruta_imagen"])
            
            concesionario_id = None
            if row["vendedor_profesional"]:
                concesionario_id = self.get_concesionario_id(row["nombre_vendedor_profesional"])
            
            data = {
                "referencia": row["referencia"],
                "peninsula_baleares": int(row["peninsula_y_baleares"]) if pd.notna(row["peninsula_y_baleares"]) else None,
                "combustible_id": combustible_id,
                "potencia": row["potencia"] if pd.notna(row["potencia"]) else None,
                "cambio_automatico": int(row["cambio_automatico"]) if pd.notna(row["cambio_automatico"]) else None,
                "carroceria_id": carroceria_id,
                "kilometraje": row["kilometraje"] if pd.notna(row["kilometraje"]) else None,
                "distintivo_ambiental_id": distintivo_ambiental_id,
                "color_id": color_id,
                "garantia": row["garantia"] if pd.notna(row["garantia"]) else None,
                "vendedor_profesional": int(row["vendedor_profesional"]) if pd.notna(row["vendedor_profesional"]) else None,
                "plazas": row["plazas"] if pd.notna(row["plazas"]) else None,
                "puertas": row["puertas"] if pd.notna(row["puertas"]) else None,
                "certificado": int(row["certificado"]) if pd.notna(row["certificado"]) else None,
                "fecha_extraccion": row["fecha_extraccion"].strftime('%Y-%m-%d %H:%M:%S'),
                "consumo": row["consumo_medio"] if pd.notna(row["consumo_medio"]) else None,
                "modelo_id": modelo_id,
                "antiguedad": anio_actual - row["anio_matricula"] if pd.notna(row["anio_matricula"]) else None,
                "precio": row["precio"] if pd.notna(row["precio"]) else None,
                "mes_matricula": row["mes_matricula"] if pd.notna(row["mes_matricula"]) else None,
                "anio_matricula": row["anio_matricula"] if pd.notna(row["anio_matricula"]) else None,
                "concesionario_id": concesionario_id,
                "url_id": url_id,
                "ruta_imagen_id": ruta_imagen_id
            }

            data = {k: v for k, v in data.items() if v is not None}
            
            query = f"""
            INSERT INTO coches_en_venta ({', '.join(data.keys())})
            VALUES ({', '.join(['%s' for _ in data.keys()])})
            """
            
            cursor = self.connection.cursor()
            cursor.execute(query, tuple(data.values()))
            self.connection.commit()
            cursor.close()


if __name__ == "__main__":
    loader = MySQLDataLoader("grupocds", "JoSanBross", "pfb_coches")
    loader.process_csv_concesionarios("/home/bross/Codigo_HAB/PFB_coches/PFB_coches/data/concesionarios_limpio.csv")
    loader.process_csv_coches("/home/bross/Codigo_HAB/PFB_coches/PFB_coches/data/coches_segunda_mano-19-02-2025_limpio.csv")